In [1]:
import tensorflow as tf
tf.enable_eager_execution()

# tensorflow/examples/tutorials/mnist/input_data.py
from tensorflow.examples.tutorials.mnist import input_data
LOGDIR ='./mnist_demo/'
mnist = input_data.read_data_sets(LOGDIR + 'data', one_hot=True)

/Users/trungtv/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist_demo/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist_demo/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./mnist_demo/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist_demo/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [4]:
import tensorflow.contrib.eager as tfe

In [29]:
def conv_layer(input, size_in, size_out):
    #kernel_shape = tf.Variable(tf.zeros([5,5, size_in, size_out]))
    kernel_shape = tfe.Variable(tf.truncated_normal([5,5, size_in, size_out], stddev=0.1))
    bias = tfe.Variable(tf.constant(0.1, shape=[size_out]))
    conv = tf.nn.conv2d(input, kernel_shape, strides=[1,1,1,1], padding='SAME')
    act = tf.nn.relu(conv + bias)
    maxpool = tf.nn.max_pool(act, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    print(tf.shape(maxpool))
    return maxpool

In [15]:
def fc_layer(input, size_in, size_out):
    #w = tf.Variable(tf.constant(0.0, shape=[size_in, size_out]))
    w = tfe.Variable(tf.truncated_normal([size_in, size_out], stddev=0.1), name="W")
    b = tfe.Variable(tf.constant(0.1, shape=[size_out]))
    act = tf.matmul(input, w) + b # NOT tf.nn.sigmoid?
    return act

In [17]:
def loss_function(logits, labels):
    xent = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=labels))
    return xent

In [40]:
# MAKING THE GRAPH
#sess = tf.Session()
#with tfe.GradientTape() as g:
x = tfe.Variable(tf.truncated_normal([100, 28 * 28 * 1])) #because we used read_data_sets
y = tfe.Variable(tf.truncated_normal([100, 10]))
x_image = tf.reshape(x, [-1, 28, 28, 1])

conv1 = conv_layer(x_image, 1, 32)
conv2 = conv_layer(conv1, 32, 64)
conv2_reshape = tf.reshape(conv2, [-1, 7 * 7 * 64])
fc1 = fc_layer(conv2_reshape, 7 * 7 * 64, 1024)
fc1_relu = tf.nn.relu(fc1)

fc2 = fc_layer(fc1_relu, 1024, 10)
# compute the cross entropy (xent)

#correct_prediction = tf.equal(tf.argmax(fc2,1), tf.argmax(y,1))
#accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
# define our optimizer
optimizer = tf.train.AdamOptimizer(1e-4)
grad_fn = tfe.implicit_value_and_gradients(loss_function)
loss_batch, grads = grad_fn(fc2, y)
optimizer.apply_gradients(grads)
print(loss_batch)

tf.Tensor([100  14  14  32], shape=(4,), dtype=int32)
tf.Tensor([100   7   7  64], shape=(4,), dtype=int32)
tf.Tensor(-1.527482, shape=(), dtype=float32)


In [41]:
#sess.run(tf.global_variables_initializer())
# train for 1000 steps
for i in range(1000):
    batch = mnist.train.next_batch(100)
    x = batch[0]
    y = batch[1]
    loss_batch, grads = grad_fn(fc2, y)
    optimizer.apply_gradients(grads)

    #
    _, xent_value = sess.run([optimizer, xent], feed_dict={x: batch[0], y: batch[1]})
    if i % 50 == 0:
        print("Step {} Xent = {}".format(i,loss_batch))
        #[train_accuracy] = sess.run([accuracy], feed_dict={x: batch[0], y: batch[1]})
        #print("Step {} Accuracy = {}".format(i, train_accuracy))

ValueError: No trainable variables were accessed while the function was being computed.

In [33]:
print(fc2)

tf.Tensor(
[[-1.88363857e+01  2.12895927e+01  5.62842131e+00  2.79343081e+00
  -1.94455605e+01  1.68308392e+01  1.31122818e+01  7.46283817e+00
  -7.28678107e-01  1.44951515e+01]
 [-2.25102558e+01  2.43134060e+01  4.84078264e+00  9.71209168e-01
  -1.57319641e+01  2.54045448e+01  1.20905514e+01  6.65793300e-01
   2.40339065e+00  1.64929638e+01]
 [-1.81080627e+01  1.81512871e+01  5.83779931e-01 -5.49387026e+00
  -2.33910599e+01  2.15979805e+01  1.62356396e+01  1.56221211e+00
   2.54836488e+00  1.12805576e+01]
 [-1.83179817e+01  1.96291409e+01  7.63982311e-02 -4.77513456e+00
  -1.90844860e+01  1.99060707e+01  1.06415195e+01  1.29962816e+01
  -3.73727679e+00  5.91949511e+00]
 [-2.27822933e+01  1.96428432e+01 -2.35017920e+00 -8.26772094e-01
  -1.49190559e+01  1.38481913e+01  8.00359154e+00 -3.70022631e+00
   3.20710230e+00  9.12756062e+00]
 [-1.85155964e+01  1.64896355e+01 -5.57669044e-01 -2.39880705e+00
  -2.26814384e+01  1.70951519e+01  9.49913597e+00 -8.02090049e-01
  -5.77700853e+00  1.3

In [38]:
tf.trainable_variables()

[]